In [1]:
import sys
sys.path.append('../src')

In [2]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

## Test vision recorder

In [ ]:
import gta.vision

In [ ]:
recorder = gta.vision.VisionRecorder()
recorder.start()
time.sleep(2)
recorder.stop()

In [ ]:
recorder.results.shape

In [ ]:
print('Mean time between images is %g sec.' % np.mean(np.diff(recorder.times)))

In [ ]:
fig, ax = plt.subplots()
i = -1
ax.imshow(recorder.results[i])
times = recorder.times
ax.set_title('$t=%g$ sec' % (times[i] - times[0],));

In [ ]:
recorder.results.shape

## Test keyboard recorder

In [ ]:
import gta.keyboard
import gta.eventIDs
reload(gta.keyboard);
reload(gta.eventIDs);

In [ ]:
recorder = gta.keyboard.KeyboardRecorder()
recorder.start()
print('Use the keyboard now ...', end=' ')
time.sleep(10)
recorder.stop()
print('done.')

In [ ]:
len(recorder.resultsList)

In [ ]:
t0 = recorder.times[0]
[
    print('%.3f %s %s' % (t-t0, eid, down))
    for t, (eid, down) in zip(recorder.times, recorder.results)
];

In [ ]:
assert -1 not in recorder.results[:, 0]

In [ ]:
recorder.results.shape

## Test gamepad recorder

In [ ]:
import gta.gamepad, gta.eventIDs
reload(gta.gamepad);
reload(gta.eventIDs);

In [ ]:
recorder = gta.gamepad.GamepadRecorder()

In [ ]:
recorder.start()

In [ ]:
recorder.worker.is_alive()

In [ ]:
allKeys = set()
nbuttons = 20
print('Use all %d controls:' % nbuttons)
while True:
    time.sleep(1)
    l = set([n for (t, (n, v)) in recorder.resultsList])
    newKeys = l.difference(allKeys)
    done = False
    if len(newKeys) > 0:
        for eid in newKeys:
            print(eid, end='=')
            print(gta.eventIDs.eids2names[eid], end='')
            allKeys.add(eid)
            if len(allKeys) >= nbuttons:
                print('.')
                done = True
            else:
                print(', ')
    if done:
        break

In [ ]:
len(recorder.times)

In [ ]:
recorder.results.shape

In [ ]:
t0 = recorder.times[0]
for (t, (eid, v)) in recorder.resultsList:
    print('%g: %s %s' % (t-t0, gta.eventIDs.eids2names[eid], v))

In [ ]:
recorder.stop()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
t0 = min(recorder.times)
for name in gta.eventIDs.keys2names['gamepad axes'].keys():
    tv = np.array([
        (t-t0, v)
        for (t, (k, v)) in zip(recorder.times, recorder.results)
         if gta.eventIDs.eids2keys[k] == name
    ])
    if len(tv) > 0:
        t = tv[:, 0]; v = tv[:, 1]
        ax.plot(t, v, label=name, marker='o', markersize=2)
ax.set_xlabel('$t-t_0$ [sec]')
ax.set_ylabel('value')
ax.legend();

## Combine recorders in one object.

In [91]:
import gta.unifiedRecorder
reload(gta.unifiedRecorder);
reload(gta.eventIDs);

In [87]:
unifiedRecorder = gta.unifiedRecorder.UnifiedRecorder(includeKeyboard=True)

unifiedRecorder.start()
print('Drive around with keyboard and controller ...', end=' ')
time.sleep(20)
unifiedRecorder.stop()
print('done.')

X, YatX, Tx = unifiedRecorder.XYT()
X.shape, YatX.shape, Tx.shape

Drive around with keyboard and controller ... done.


((435, 797, 1366, 3), (435, 143), (435,))

What controls were held at a few random times?

In [88]:
for ix in 0, 42, -60:
    print('t = %.3gs: %s' % (
        Tx[ix]-Tx[0],
        [
            gta.eventIDs.eids2names[eid]
            for eid, value in enumerate(YatX[ix]) if value
        ])
    )

t = 0s: []
t = 3.32s: ['gamepad_l_thumb_x', 'gamepad_l_thumb_y', 'gamepad_right_trigger']
t = 27.6s: ['gamepad_l_thumb_x', 'gamepad_l_thumb_y', 'keyboard_left shift']


How frequent was holding down multiple controls simultaneously?

In [ ]:
numKeys = YatX.astype(bool).sum(1)
numKeysSet = np.array(sorted(list(set(numKeys))))
counts = [
    (numKeys == n).sum()
    for n in numKeysSet
]
fig, ax = plt.subplots()
ax.bar(numKeysSet, counts)
ax.set_xlabel('number of simultaneous controls applied')
ax.set_ylabel('count of image frames');

Frequent, but some of this might have been from the fact that the left stick was apparently stuck away from zero but within its dead zone while keyboard controls were being used. The frequent 3-control states probably were nonzero values of left stick x/y and right trigger (for steering and throttle).

### Record without keyboard.
Note that there are fewer columns in `YatX`.

In [93]:
unifiedRecorder = gta.unifiedRecorder.UnifiedRecorder(includeKeyboard=False)

unifiedRecorder.start()
print('Drive around with controller only ...', end=' ')
time.sleep(20)
unifiedRecorder.stop()
print('done.')

X, YatX, Tx = unifiedRecorder.XYT()
X.shape, YatX.shape, Tx.shape

Drive around with controller only ... done.


((264, 797, 1366, 3), (264, 20), (264,))